# SentenceTransformers SBERT

https://www.sbert.net/index.html


## getting started

In [25]:
import numpy as np
import pandas as pd
import pickle

from sentence_transformers import SentenceTransformer, util

In [2]:
## Mult-lingual model that supports Dutch
modelML = SentenceTransformer('xlm-r-100langs-bert-base-nli-mean-tokens')

## import kamer speeches

In [5]:
%%time
tweede_kamer = pd.read_csv("CorpusTweedeKamer.zip")
tweede_kamer = (
    tweede_kamer
    .assign(datum = pd.to_datetime(tweede_kamer.date))
    .assign(speaker = tweede_kamer.speaker.str.lower())
)

CPU times: user 12.9 s, sys: 1 s, total: 13.9 s
Wall time: 14.7 s


In [6]:
recente_speeches = (
    tweede_kamer
    .dropna(subset = ["text"])
    .query('terms > 75')
    .query('terms < 800')
    .query('datum > "2016-01-01"')
)

In [18]:
recente_speeches.shape

(95314, 12)

In [19]:
speeches = recente_speeches.text.str.lower().values.tolist()

In [22]:
%%time 
sentence_embeddings = modelML.encode(speeches)

CPU times: user 47min 22s, sys: 1min 44s, total: 49min 6s
Wall time: 12min 51s


In [23]:
sentence_embeddings.shape

(5000, 768)

In [26]:
#Store sentences & embeddings on disc
with open('kamer_BERT_embeddings.pkl', "wb") as fOut:
    pickle.dump({'sentences': speeches[0:5000], 'embeddings': sentence_embeddings}, fOut, protocol=pickle.HIGHEST_PROTOCOL)



In [ ]:
#Load sentences & embeddings from disc
with open('kamer_BERT_embeddings.pkl', "rb") as fIn:
    stored_data = pickle.load(fIn)
    stored_sentences = stored_data['sentences']
    stored_embeddings = stored_data['embeddings']

### Voer een semantic search uit

In [56]:
query_embedding = modelML.encode("de belastingdienst functioneert niet goed", convert_to_tensor=True)
topmatches = util.semantic_search(query_embedding, sentence_embeddings)

In [57]:
topmatches

[[{'corpus_id': 2823, 'score': 0.68560827},
  {'corpus_id': 3316, 'score': 0.6840711},
  {'corpus_id': 1104, 'score': 0.6724966},
  {'corpus_id': 1511, 'score': 0.6650846},
  {'corpus_id': 1461, 'score': 0.663343},
  {'corpus_id': 3518, 'score': 0.65895706},
  {'corpus_id': 4441, 'score': 0.65877104},
  {'corpus_id': 4577, 'score': 0.65877104},
  {'corpus_id': 4451, 'score': 0.6503725},
  {'corpus_id': 4587, 'score': 0.6503725}]]

In [58]:
top_textst = [ idx['corpus_id']  for idx in topmatches[0]]
top_textst

[2823, 3316, 1104, 1511, 1461, 3518, 4441, 4577, 4451, 4587]

In [59]:
[speeches[0:5000][i] for i in top_textst]

['dat is geen zinvolle vergelijking, want de kern van de situatie nu is dat we eigenlijk niet in staat zijn tot een redelijke handhaving. dan bestaat er dus ook geen wezenlijke, omvangrijke naheffing. eerlijk gezegd zou mijn idee voor de nieuwe situatie zijn dat we niet genoodzaakt zijn om uitgebreid na te heffen. het uiteindelijke doel moet immers zijn om schijnconstructies te voorkomen en niet om ze te laten voorkomen en dan te moeten beboeten. op dit moment ligt de handhaving op een zeer laag pitje omdat wij eenvoudigweg niet in staat zijn om te handhaven. in de nieuwe situatie willen wij liever voorkomen dan bekeuren; zo zou ik het willen uitdrukken.',
 'ik stel voorop dat ik de groep wilsonbekwamen niet zou willen rangschikken onder de mensen die niet kunnen lezen en schrijven. u noemt die groepen in één adem. dat vind ik echt onbegrijpelijk. als mensen niet kunnen lezen en schrijven of als zij de taal niet eigen zijn, hebben wij in nederland allerlei manieren om te communiceren. 